# Web Scraping Project

## 

In [25]:
# Import packages for scraping/processing data
import requests
from bs4 import BeautifulSoup
import re
import numpy

In [4]:
ds_2019 = 'https://old.reddit.com/r/datascience/comments/e8fown/official_2019_end_of_year_salary_sharing_thread/?limit=500'
ds_2020 = 'https://old.reddit.com/r/datascience/comments/klvb55/official_2020_end_of_year_salary_sharing_thread/?limit=500'

ds_2019_response = requests.get(ds_2019)

In [5]:
page = requests.get(ds_2019)
print(page)

<Response [429]>


In [6]:
print(ds_2019_response)

<Response [200]>


In [8]:
ds_2019_page_content = BeautifulSoup(ds_2019_response.text, 'html.parser')

In [36]:
# print(ds_2019_page_content.prettify())

## com

In [168]:
ds_2019_comments_div = ds_2019_page_content.select(".md")
# ds_2019_comments_div[0]
has_record_bool = list()
for comment in ds_2019_comments_div:
    if re.search('Title:', str(comment)):
        has_record_bool.append(True)
    else:
        has_record_bool.append(False)

# print(has_record_bool[0:6])
# record_filter = [i for i, x in enumerate(has_record_bool) if x]
# print(record_filter)
ds_2019_salary_records = [i for (i, v) in zip(ds_2019_comments_div, has_record_bool) if v]
ds_2019_salary_records_dicts = list()
record_number = 0
for record in ds_2019_salary_records[1:]:
    record_number += 1
    record_dict = {}
    record = re.sub(r'\<\/?strong\>', '', str(record))
    record = re.sub(r'\<\/?p\>', '', str(record))
    record = re.sub(r'\<\/?ul\>', '', str(record))
    record = re.sub(r'(<li>Prior Experience\:)\s?\n+\<\/?li\>([A-z0-9\s\(\=\)\:]+)\<\/?li\>\n+\<\/?li\>([A-z0-9\s\(\=\)]+)\<\/?li\>\n+\<\/?li\>([A-z0-9\s\(\=\)]+)\<\/?li\>\n+', r'\1 \2, \3, \4', str(record))
    record = re.sub(r'(<li>Prior Experience\:)\s?\n+\<\/?li\>([A-z0-9\s\(\=\)\:]+)\<\/?li\>\n+\<\/?li\>([A-z0-9\s\(\=\)]+)\<\/?li\>\n+', r'\1 \2, \3', str(record))
    record = re.sub(r'(<li>Prior Experience\:)\s?\n+\<\/?li\>([A-z0-9\s\(\=\)\:]+)\<\/?li\>\n+', r'\1 \2', str(record))
    record = re.sub(r'\$(Internship|Coop)\s([A-z0-9]+)', r'\1: \2', str(record))
    record = re.sub(r'\$(Internship|Coop)(<\/li>)', r'\1: None\2', str(record))
    record = re.sub(r'\$(Internship|Coop)', r'\1', str(record))
    record = re.sub(r'(remote)\:', r'\1', str(record))
    record = re.sub(r'([Cc]ompany\/)', r'', str(record))
    record = re.sub(r'([Cc]ompany):', r'Industry', str(record))
    record = re.sub(r'(BS):', r'\1', str(record))
    record = re.sub(r'(Data Analyst):', r'\1', str(record))
    record = re.sub(r'(Data Scientist):', r'\1', str(record))
    record = re.sub(r':( Aviation)', r'\1', str(record))
#     record = re.sub(r'(Analyst):', r'\1', str(record))
    record = re.sub(r'(<li>[A-Za-z0-9\s\$\/\-\~,\'\(\)\|\=\.\+\%\;]+)\s?([A-Za-z0-9\s\$\/\-\~,\'\(\)\|\=\.\+\%\;]+<\/li>\n?)', r'', str(record))
#     record = re.sub(r'', r'', str(record))
    print(record)
    record_title = re.search('[Tt]itle:\s?([A-z0-9\s\(\)\/\-\.\,\%]+)[<\n]', record)
    record_tenure = re.search('[Tt]enure(?: [Ll]ength)?:\s?([A-z0-9\s\(\)\/\-\.\,\%\'\;\&]+)[<\n]', record)
    if record_tenure == None:
        record_tenure = ['filler', 'none given']
    record_location = re.search('[Ll]ocation:\s?([A-z0-9\s\(\)\/\-\.\,\%\'\;]+)[<\n]', record)
    record_salary = re.search('(?:[Ss]alary|[Bb]ase):\s?([A-z0-9\s\(\)\/\-\.\,\%\'\;\$\€\~\£\+]+)[<\n]', record)
    record_industry = re.search('[Ii]ndustry:?\s?([A-z0-9\s\(\)\/\-\.\,\%\'\;\$\€\~\£\+\&]+)[<\n]', record)
    if record_industry == None:
        record_industry = ['filler', 'none given']
    record_education = re.search('[Ee]ducation:?\s?([A-z0-9\s\(\)\/\-\.\,\%\'\;\$\€\~\£\+\&\|\’]+)[<\n]', record)
    if record_education == None:
        record_education = ['filler', 'none given']
    record_prior_experience = re.search('(?:[Pp]rior)?\s?(?:[Ee]xperience|[Ee]xp):?\s?([A-z0-9\s\(\)\/\-\.\,\%\'\;\$\€\~\£\+\&\=\@\"]+)[<\n]', record)
    if record_prior_experience == None:
        record_prior_experience = ['filler', 'none given']
    record_total_comp = re.search('[Tt]otal (?:[Cc]omp|[Cc]ompensation):?\s?([A-z0-9\s\(\)\/\-\.\,\%\'\;\$\€\~\£\+\&\;\?]+)[<\n]', record)
    if record_total_comp == None:
        record_total_comp = ['filler', 'none given']
    print('\n',record_total_comp[1], '\n')
#     record_dict['title'] = record_title[1]
#     record_dict['tenure'] = record_tenure[1]
#     record_dict['location'] = record_location[1]
#     record_dict['salary'] = record_salary[1]
#     record_dict['industry'] = record_industry[1]
#     record_dict['education'] = record_education[1]
#     record_dict['prior_experience'] = record_prior_experience[1]
    record_dict['total_comp'] = record_total_comp[1]
    record_elements = re.findall(r'(?:<li>|<p>)([A-Za-z0-9\s\:\$\/\-\~,\'\(\)\|\=\.\+\%\;]+)(?:<\/li>|<\/p>)', record)
#     print(record_elements)
#     for element in record_elements:
#         c, d = element.split(':', 1)
#         record_dict[c] = d.strip()
    ds_2019_salary_records_dicts.append(record_dict)
#     print(record_dict)
print(ds_2019_salary_records_dicts)
print(len(ds_2019_salary_records_dicts))
print(len(ds_2019_salary_records))

<div class="md">
<li>Title: ML Engineer (L7/E7-equivalent level)</li>
<li>Tenure length: 1year</li>
<li>Location: Seattle</li>
<li>Salary: $295k</li>
<li>Industry: SaaS</li>
<li>Education: MS</li>
<li>Prior Experience: ~15yrs, several FAANG-ish companies</li>
<li>Relocation/Signing Bonus: $75k</li>
<li>Stock and/or recurring bonuses: $75k bonus, $270k RSU's per year</li>
<li>Total comp: $640k</li>

</div>

 $640k 

<div class="md">Throwaway because others know my main acct.

<li>Title: Manager Data Analytics/Science</li>
<li>Tenure length: 1yr</li>
<li>Location: Pennsylvania LCOL area (not PHI/PIT)</li>
<li>Salary: $160,000</li>
<li>Industry: Utility</li>
<li>Education: BS Risk Analytics | MS Data Science</li>
<li>Prior Experience: 9 yrs in various analytical roles (data, intelligence, and data scientist for last 3ish yrs) for federal government.</li>
<li>Stock: ~$20k-50k </li>
<li>recurring bonuses: ~$20k-50k</li>
<li>Total comp: ~$200k-260k</li>

Evolved into 'data scientist' as many